In [36]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn.model_selection import cross_val_score

data_raw = pd.read_csv("VCG_Data_Clean.csv", encoding='ISO 8859-1')

data = data_raw[list(data_raw.loc[:,'gender':'race']) + ['religion'] + 
         list(data_raw.loc[:,'biden_vir_humil':'trump_sin_sloth']) + 
         list(data_raw.loc[:,'party':'biden_hill']) + ['shift']]

x = data.copy()

x = x.drop(['trump_why_text','lean'], axis = 1)

xtot = x.dropna(subset=['shift'])

y = xtot['shift']

x = xtot.drop(['shift'], axis=1)

num_col = x.select_dtypes(['number']).columns

imp = SimpleImputer(strategy='mean')
imp.fit(x[num_col])
x[num_col] = imp.transform(x[num_col])

cat_col = x.select_dtypes(['object']).columns

xint = x.copy()

xint[cat_col] = x[cat_col].astype('category')

xint[cat_col] = xint[cat_col].apply(lambda x: x.cat.codes)

# How many shifters?

print(y.value_counts())

# Create train/test splits

x_train, x_test, y_train, y_test = train_test_split(xint, y, test_size = 0.2, random_state = 7)




stay     371
shift     67
Name: shift, dtype: int64


In [39]:
xtot['shift'].groupby(xtot['gender']).describe()

,count,unique,top,freq
gender,,,,
Female,253,2,stay,214
Male,185,2,stay,157


In [30]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB().fit(xint, y)

scores = cross_val_score(nb, xint, y, cv=10)
scores

#pred = nb.predict(x_test)
#print(np.mean(pred == y_test))

array([0.95454545, 0.97727273, 0.95454545, 0.86363636, 0.97727273,
       0.95454545, 0.95454545, 0.97727273, 1.        , 0.93023256])

In [32]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, pred))

xint.groupby(xint['shift'])

[[12  0]
 [ 3 73]]


KeyError: 'shift'

In [29]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = 2)
gmm.fit(xint)

labels = gmm.predict(xint)
frame = xint.copy()
frame['shift'] = labels

frame.head()

,gender,age_range,state,hispanic,race,religion,biden_vir_humil,biden_vir_kind,biden_vir_modest,biden_vir_chaste,...,biden_why_moral,biden_why_honest,biden_why_virus,biden_why_race,biden_why_immig,biden_why_poor,biden_why_econ,biden_why_foreign,biden_hill,shift
0,1,3,1,0,8,1,3.000,3.000000,4.00,3.00000,...,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3,1
1,0,3,4,0,8,1,0.000,1.000000,2.00,2.00000,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
2,0,3,3,0,8,1,1.000,2.000000,1.00,1.21148,...,3.0,1.0,4.0,3.0,0.0,3.0,0.0,3.0,3,1
4,1,2,0,0,8,0,0.000,0.000000,0.00,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0
5,0,3,4,0,8,0,1.024,1.446575,1.06,1.21148,...,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0,1
